In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('data/CCPP/Folds5x2_pp.xlsx')

https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

In [ ]:
print open('data/CCPP/Readme.txt').read()

In [ ]:
data.describe()

In [ ]:
data.cov()

In [ ]:
%matplotlib inline

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data.plot.scatter(x = 'V', y = 'PE')

In [ ]:
data[(15 < data.AT) & (data.AT < 25)].plot.scatter(x = 'V', y = 'PE')

In [ ]:
X = data[(15 < data.AT) & (data.AT < 25)][['V']]

In [ ]:
y = data[(15 < data.AT) & (data.AT < 25)].PE

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
X_train.shape, y_train.shape

In [ ]:
r5 = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_train)

In [ ]:
sum((y_test - r5.predict(X_test))**2) / len(y_test)

In [ ]:
sum((y_test - y_test.mean())**2) / len(y_test)

In [ ]:
def SPE(f):
    return sum((y_test - f.predict(X_test))**2) / len(y_test)

In [ ]:
ks = range(1, 21)
models = [KNeighborsRegressor(n_neighbors=i).fit(X_train, y_train) for i in ks]
SPEs = [SPE(f) for f in models]

In [ ]:
plt.plot(ks, SPEs, "o")
plt.xlabel('k')
plt.ylabel('SPE')

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
f_lr = LinearRegression().fit(X_train, y_train)

In [ ]:
SPE(f_lr)